<a href="https://colab.research.google.com/github/Qv-code-Qv/kayak_data/blob/main/booking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scrapy

In [ ]:
!python booking.py

In [ ]:
import plotly.io as pio
#pio.renderers.default = "svg" #to uncomment with collab use

In [2]:
import plotly.express as px
import pandas as pd

In [40]:
df = pd.read_csv('gps_data_1.csv')
df.head()
   
fig = px.scatter_mapbox(df, opacity=1, title="Our best destinations for the coming week :", color="Ville", text="Hotel",  lat="Latitude", lon="Longitude",
                        zoom=4, mapbox_style="open-street-map")
fig.show()

In [1]:
#!pip install folium matplotlib mapclassify
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

hostels_csv = pd.read_csv('gps_data.csv')
carte =gpd.GeoDataFrame(hostels_csv, geometry = gpd.points_from_xy(hostels_csv['Longitude'], hostels_csv['Latitude'])).drop(['Longitude' , 'Latitude'], axis = 1)


ModuleNotFoundError: No module named 'geopandas'